# Turma #867   -   Projeto Machine Learning II (parte II - Pipelines)
#### Equipe 4:
- Adriana Roberta Miceli de Souza <br/>
- Debora Kassem Buturi <br/>
- Helen Cristina de Acypreste Rocha <br/> 
- Marcus Fontes <br/>
- Richard Raphael Banak <br/>

## Entregáveis

- Dois notebooks: (i) com a investigação e comparações feitas; (ii) com o fluxo limpo do modelo escolhido.
- Crie pipeline para o modelo.
- Realize processo de otimização dos hiperparâmetros.
- Utilize pelo menos uma variável categórica e pelo menos uma variável numérica.
- Para garantir robustez, adicione uma camada de imputação de missing para todas as features. Reflita sobre qual é a melhor estratégia para cada uma das variáveis explicativas.
- Não foque mais em utilizar apenas 5 variáveis, vamos abrir os horizontes e buscar outras features que podem ser úteis. Vale salientar que a depender do modelo, não adianta simplesmente colocar todas porque isso pode gerar problemas de overfitting, de underfitting ou de eficiência computacional, então será necessário fazer algum tipo de seleção.
- Apenas um modelo será entregue. Faça a escolha baseando-se em performance, mas levando em consideração custo computacional. Descreva o processo de decisão e argumente a favor do modelo - deve estar dentro do fluxo limpo, no notebook modelo escolhido, no início.
- Para marcar a previsão, use a probabilidade do evento (sem binarizar).
- **Meta em bater pelo menos 0.70 de AUC. Se não conseguir, não há problema, mas falar antecipadamente com Rychard/Bruno.**

## Modelo escolhido e argumentos

- Foi utilizada a técnica de XGBOOST por ter os melhores resultados em termos ROC AUC. Apenas com a seleção de variáveis foi possível obter um ROC AUC de 70% (na amostra de teste), antes mesmo de qualquer otimização de hiperparâmetros, sem um custo computacional muito superior aos outros modelos. Foi possível paralelizar a execução desse modelo através do parâmetro n_jobs que, neste caso concreto, performou melhor se configurado com o valor 16, frente às outras tentativas que fizemos (2 a 64). 
<br/> 

- Durante os testes com diversos conjuntos de variáveis foi possível observar que esta modelagem é bastante "sensível" a overfitting, especialmente com excesso de variáveis. Para contornar o problema precisariamos de algum tipo de regularização e de um conjunto menor de variáveis explicativas, além de testar os parâmetros específicos como lambda, alpha e eta/learning_rate. 
<br/>

- O tempo de processamento do gridsearch com o xgboost durante a otimização dos hiperparâmetros, mesmo com o uso de n_jobs = 16, mostrou-se bastante grande quando a combinação de parâmetros ultrapassa uma certa quantidade de elementos (questão exponencial do gridsearch).
<br/>
- O modelo final contou com 25 variáveis que foram escolhidas através de critérios de exclusão como % de missing do campo (>= 10%), variáveis com domínio praticamente único (+ que 99% de concentração), além de anáise manual/visual da correlação das variáveis explicativas. Também foi utilizada a ferramenta SelectKBest para escolha das variáves. 

# 0. Imports

!pip install xgboost

In [6]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder,LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import xgboost as xgb

### 1. Leitura dos dados

In [7]:
caminho = './projeto_ml2/'
arquivo_principal = 'application_train.csv'
arquivo_oculto = 'application_test_student.csv'
arquivo_metadados = 'HomeCredit_columns_description.csv'

df = pd.read_csv(f'{caminho}/{arquivo_principal}')
df_oculto = pd.read_csv(f'{caminho}/{arquivo_oculto}')
df_metadados = pd.read_csv(f'{caminho}/{arquivo_metadados}', encoding = 'Windows-1252')

### 2. Aplicação do melhor modelo

2.1 Declaração de variáveis

In [17]:
var_resp = 'TARGET'

var_string_binaria = ['CODE_GENDER','FLAG_OWN_REALTY','NAME_CONTRACT_TYPE','FLAG_OWN_CAR']
var_string_nao_binaria = ['NAME_EDUCATION_TYPE','NAME_HOUSING_TYPE','NAME_INCOME_TYPE']
var_string = var_string_binaria + var_string_nao_binaria

var_num_discretas = ['CNT_CHILDREN','DEF_30_CNT_SOCIAL_CIRCLE','REGION_RATING_CLIENT']
var_num_continuas = ['AMT_INCOME_TOTAL','DAYS_REGISTRATION','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE','EXT_SOURCE_2','REGION_POPULATION_RELATIVE']
var_num_flag = ['FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_18'] 
var_num = var_num_discretas + var_num_continuas + var_num_flag
var_num_sem_flag = var_num_discretas + var_num_continuas

var_expl = var_num + var_string

### 3. Train x Test Split

In [18]:
df_treino, df_teste = train_test_split(df, test_size = 0.3, random_state = 10)

In [19]:
x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()
y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

### 4. Aplicação do melhor modelo com Pipelines

4.1 Listas ordenadas

In [15]:
lista_ordenada_NAME_EDUCATION_TYPE = [
    
    'Lower secondary',
    'Secondary / secondary special', 
    'Incomplete higher',
    'Higher education', 
    'Academic degree' 
]
oe_NAME_EDUCATION_TYPE = OrdinalEncoder(categories = [lista_ordenada_NAME_EDUCATION_TYPE])

lista_ordenada_NAME_HOUSING_TYPE = [
    
    'With parents',
    'Municipal apartment',
    'Co-op apartment',
    'Rented apartment',
    'Office apartment',
    'House / apartment'
]
oe_NAME_HOUSING_TYPE = OrdinalEncoder(categories = [lista_ordenada_NAME_HOUSING_TYPE])

lista_ordenada_NAME_INCOME_TYPE = [

'Unemployed',
'Student',
'Maternity leave',
'Pensioner',
'Working',
'Commercial associate',
'Businessman',            
'State servant'
    
]
oe_NAME_INCOME_TYPE = OrdinalEncoder(categories = [lista_ordenada_NAME_INCOME_TYPE])

4.2 Pipelines de pré-processamento e aplicação do modelo

In [37]:
pipeline_imput_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

pipeline_input_flg = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first',sparse=False))
])

pipeline_NAME_EDUCATION_TYPE = Pipeline(steps=[
    ('NAME_EDUCATION_TYPE', oe_NAME_EDUCATION_TYPE)    
])

pipeline_NAME_HOUSING_TYPE = Pipeline(steps=[
    ('NAME_HOUSING_TYPE', oe_NAME_HOUSING_TYPE)    
])

pipeline_NAME_INCOME_TYPE = Pipeline(steps=[
    ('NAME_INCOME_TYPE', oe_NAME_INCOME_TYPE)    
])

pipeline_preproc = ColumnTransformer(transformers = [
    ('numericas',            pipeline_imput_num,             var_num_sem_flag        ),
    ('strings',              pipeline_input_flg,             var_string              ),
    ('NAME_EDUCATION_TYPE',  pipeline_NAME_EDUCATION_TYPE,  ['NAME_EDUCATION_TYPE']  ),
    ('NAME_HOUSING_TYPE',    pipeline_NAME_HOUSING_TYPE,    ['NAME_HOUSING_TYPE']    ),
    ('NAME_INCOME_TYPE',     pipeline_NAME_INCOME_TYPE,     ['NAME_INCOME_TYPE']     )
])

melhor_modelo = xgb.XGBClassifier(    
    max_depth = 5,
    learning_rate = 0.2,
    #lambda = 0.5,
    n_jobs= 16,
    random_state = 1    
)

pipeline_final =  Pipeline(steps=[('preproc_modelo', pipeline_preproc) , ('melhor_modelo', melhor_modelo)])

4.3 Ajuste do melhor modelo com pipelines (FIT)

In [38]:
pipeline_final.fit(x_treino,y_treino)

y_pred_treino = pipeline_final.predict_proba(x_treino)[:, 1]
y_pred_teste = pipeline_final.predict_proba(x_teste)[:, 1]

roc_auc_score(y_treino, y_pred_treino) , roc_auc_score(y_teste, y_pred_teste)

(0.7734857175889174, 0.7111350811483095)

### 5. PREDICT

5.1 Adaptando os dados de aplicação (df_oculto)

In [27]:
df_oculto = df_oculto[(df_oculto['CODE_GENDER'] != 'XNA') | (df_oculto['NAME_FAMILY_STATUS'] != 'Unknown')]

5.2 - Aplicação do melhor modelo e Export dos resultados para arquivo csv

In [40]:
df_oculto['Y_PRED'] = pipeline_final.predict_proba(df_oculto)[:, 1]
df_oculto[['SK_ID_CURR','Y_PRED']].to_csv('previsao_credito_equipe4_MLII.csv', index=False)
df_oculto[['SK_ID_CURR','Y_PRED']].head()

,SK_ID_CURR,Y_PRED
0,149741,0.104982
1,363290,0.085853
2,436006,0.030565
3,377703,0.038150
4,188624,0.303696
